<a id='h0'></a>
## Рекомендация тарифов

<h1>Содержание (кликабельное):</h1>
<a href='#h0'>1. Название прокта</a><br>
<a href='#h1'>2. Описание проекта, импорт библиотек</a><br>
&nbsp;&nbsp;&nbsp;<a href='#h2'>2.1. Импорт библиотек</a><br>
&nbsp;&nbsp;&nbsp;<a href='#h3'>2.2 Ознакомление с данными</a><br>
<a href='#h4'>3. Разбейте данные на выборки</a><br>
<a href='#h5'>4. Исследуйте модели</a><br>
&nbsp;&nbsp;&nbsp;<a href='#h51'>4.1. Дерево решений,</a><br> 
&nbsp;&nbsp;&nbsp;<a href='#h52'>4.2  Случайный лес</a><br>
&nbsp;&nbsp;&nbsp;<a href='#h53'>4.3. Логистическая регрессия</a><br>
&nbsp;&nbsp;&nbsp;<a href='#h54'>4.4. Вывод</a><br>
<a href='#h6'>5. Проверьте модель на тестовой выборке</a><br>
&nbsp;&nbsp;&nbsp;<a href='#h61'>5.1. Вывод</a><br>
<a href='#h7'>6. (бонус) Проверьте модели на адекватность</a><br>
<a href='#h8'>7. Чек-лист готовности проекта
<a href='#vidod'>8. Вывод</a><br>


<a id='h1'></a>
## Изучим общую информацию.

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».<br>
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.<br>
Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

**Описание данных**<br>
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:<br>
сalls — количество звонков,<br>
minutes — суммарная длительность звонков в минутах,<br>
messages — количество sms-сообщений,<br>
mb_used — израсходованный интернет-трафик в Мб,<br>
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).<br>

<a id='h2'></a>
### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

pd.set_option('display.max_columns', None)
import warnings 
warnings.filterwarnings("ignore")


<a id='h3'></a>
### Ознакомление с данными

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Вывод

предстаавлена задача классификации. <br>
Целевой признак  is_ultra. <br>
Остальные признаки помогут искать решения для is_ultra.<br>

<a id='h4'></a>
## Разбейте данные на выборки

Определим features (признаки) и target (целевой признак).

In [5]:
features = data.drop(['is_ultra'], axis=1)
target = data["is_ultra"]

In [6]:
df_train, df_other = train_test_split(data, test_size=0.4, random_state=12345,stratify = data['is_ultra'])
df_test, df_valid = train_test_split(df_other, test_size=0.5, random_state=12345,stratify = df_other['is_ultra'])

Определим features (признаки) и target (целевой признак) для выборок

In [7]:
#тренировочная
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train["is_ultra"]
#валидная
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid["is_ultra"]
#тестовая
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test["is_ultra"]

Вывод

Данные были разбиты по принципу 60/20/20.

Обучение train 
Валидация valid
Лучшая модель по валидации будет проверена на данных test

In [8]:
features_train.shape

(1928, 4)

<a id='h5'></a>
## Исследуйте модели


Исследуем модели дерево решений, случайный лес и линейную регрессию

<a id='h51'></a>
### Дерево решений

In [8]:
best_model = None
best_result = 0
best_depth = 0

for depth in range(1,30):
    model_tree = DecisionTreeClassifier(random_state = 11111, max_depth = depth, criterion='entropy')
    model_tree.fit(features_train,target_train)
    prediction_valid = model_tree.predict(features_valid)
    res = model_tree.score(features_valid, target_valid)
    #print('max_depth =',depth, 'результат = ',res )    
    if best_result < res:
        best_model = model_tree
        best_result = res
        best_depth = depth
        best_tree_res = best_result

print('Лучший результат дерева: ',best_tree_res, 'глубина',best_depth)

Лучший результат дерева:  0.8055987558320373 глубина 5


<a id='h52'></a>
### Случайный лес

In [9]:
best_model = None
best_result = 0
best_est = 0

for est in range(10,70):
        model_forest = RandomForestClassifier(random_state = 11111 ,n_estimators = est)
        model_forest.fit(features_train,target_train)
        prediction_valid = model_forest.predict(features_valid)
        res = model_forest.score(features_valid, target_valid)
        # print('est =', est, 'результат = ', res )  
        if best_result < res:
            best_model = model_forest
            best_est = est
            best_result = res
            best_forest_res = best_result

print('Лучший результат лес: ', best_forest_res, 'n_estimators=', best_est)            

Лучший результат лес:  0.8180404354587869 n_estimators= 30


<a id='h53'></a>
### Регрессию

In [10]:
best_max_iter = 0
best_result = 0
best_reg_res = 0

for max_iteration in range(1, 1000):
    model_reg = LogisticRegression(random_state=11111, solver='lbfgs', max_iter = max_iteration )
    # обучим модель перебирая гипер параметр iter
    model_reg.fit(features_train, target_train) # обучите модель на тренировочной выборке
    result = model_reg.score(features_valid,target_valid)
    #print('Результат', result, 'Лучшая итерация = ', max_iteration )    
    if result > best_result:
        best_max_iter = max_iteration
        best_result = result
        best_reg_res = best_result

print("Accuracy наилучшей модели на валидационной выборке:", best_reg_res, "Макс кол-во итераций:", best_max_iter)

Accuracy наилучшей модели на валидационной выборке: 0.7573872472783826 Макс кол-во итераций: 77


Выведем на экран результаты трех моделей

In [11]:
print('DecisionTreeClassifier', best_tree_res)
print('RandomForestClassifier', best_forest_res)
print('LogisticRegression', best_reg_res)

DecisionTreeClassifier 0.8055987558320373
RandomForestClassifier 0.8180404354587869
LogisticRegression 0.7573872472783826


<a id='h54'></a>
### Вывод

Точность модели логистической регресии на валидационной выборке: 0.7573872472783826 Макс кол-во итераций: 77 <br>
Точность модели случайный лес на валидационной выборке:  0.8180404354587869 <br>
Точность модели Дерево решений на валидационной выборке: 0.8055987558320373 глубина 5 <br>
По показателю точности, наиболее качественные результаты прогнозирует модель случайный лес (Random Forest)<br>

<a id='h6'></a>
## Проверьте модель на тестовой выборке

In [12]:
prediction_test_reg = model_reg.predict(features_test)
prediction_test_forest = model_forest.predict(features_test)
prediction_test_tree = model_tree.predict(features_test)

In [13]:
accuracy_reg_test = accuracy_score(prediction_test_reg, target_test)
accuracy_forest_test = accuracy_score(prediction_test_forest, target_test)
accuracy_tree_test = accuracy_score(prediction_test_tree, target_test)

In [14]:
print('DecisionTree', accuracy_tree_test)
print('RandomForest', accuracy_forest_test)
print('LogisticRegression', accuracy_reg_test)

DecisionTree 0.7262830482115086
RandomForest 0.7916018662519441
LogisticRegression 0.7387247278382582


<a id='h61'></a>
### Вывод

Наиболее верные предсказания делает модель случайного леса Random Forest, это видно по точности предсказания на валидационной и на тестовой выборке. 

<a id='h7'></a>
## (бонус) Проверьте модели на адекватность

In [15]:
x = data.drop(['is_ultra'], axis=1)
y = data["is_ultra"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=27)

dummy = DummyClassifier(strategy='most_frequent').fit(x_train, y_train)
dummy_pred = dummy.predict(x_test)

In [16]:
print('Test score: ', accuracy_score(y_test, dummy_pred))

Test score:  0.6932171748599876


Мы получили оценку точности 69,3% -  без обучения модели

Наши модели показывают результат выше точность модели без обучения значит их можно считать адекватными?

DecisionTreeClassifier 0.7262830482115086
RandomForestClassifier 0.7900466562986003
LogisticRegression 0.7387247278382582

Логическая регрессия

In [17]:
lr = LogisticRegression(solver='lbfgs').fit(x_train, y_train)
lr_pred = lr.predict(x_test)

In [18]:
accuracy_score(y_test, lr_pred)

0.7498444306160548

In [19]:
predictions = pd.DataFrame(lr_pred)
predictions[0].value_counts()

0    1476
1     131
Name: 0, dtype: int64

Случайный лес

In [20]:
rf = RandomForestClassifier(random_state = 11111 ,n_estimators = est).fit(x_train, y_train)
rf_pred = rf.predict(x_test)

In [21]:
accuracy_score(y_test, rf_pred)

0.7996266334785315

In [22]:
predictions = pd.DataFrame(rf_pred)
predictions[0].value_counts()

0    1260
1     347
Name: 0, dtype: int64

Дерево решений

In [23]:
dt = DecisionTreeClassifier(random_state = 11111, max_depth = depth, criterion='entropy').fit(x_train, y_train)
dt_pred = dt.predict(x_test)

In [24]:
accuracy_score(y_test, dt_pred)

0.7069072806471687

In [25]:
predictions = pd.DataFrame(dt_pred)
predictions[0].value_counts()

0    1115
1     492
Name: 0, dtype: int64

Вывод: Для дальнейшей работы следует использовать модель Случайный лес, остальные модели показывают не высокую оценку точности.

<a id='vidod'></a>
## Итоговый вывод<b>

В ходе  работы было выполнено:<br>
Изучен файл с данными.<br>
Разбиты данные на три выборки: обучающая, валидационная и тестовая.<br>
Исследованы три модели классификации: Решающее дерево, Случайный лес и Логистическая регрессия.<br>
Оценена точность обученной модели.<br>
Оценена адекватность модели.<br>

Была решена задача классификации по целевому признаку  is_ultra. <br>
Данные были разбиты по принципу 60/20/20. <br>
Точность модели логистической регресии на валидационной выборке: 0.7573872472783826 Макс кол-во итераций: 77 <br>
Точность модели случайный лес на валидационной выборке:  0.8180404354587869 <br>
Точность модели Дерево решений на валидационной выборке: 0.8055987558320373 глубина 5 <br>
По показателю точности, наиболее качественные результаты прогнозирует модель случайный лес (Random Forest)<br>
Наиболее верные предсказания делает модель случайного леса Random Forest, это видно по точности предсказания на валидационной и на тестовой выборке.<br>
Для дальнейшей работы следует использовать модель Случайный лес, остальные модели показывают не высокую оценку точности, по сравнению со случайной моделью.